In [1]:
# Initializing Spark
with open("setupPySpark.py", "r") as setup_file:
    exec(setup_file.read())


In [2]:
# Spark context
from pyspark.sql.session import SparkSession

spark = SparkSession(sc)

In [3]:
# SQL context
from pyspark.sql import SQLContext

sqlContext = SQLContext(sc)

In [4]:
# Importing required functions
from util import read_file, read_folder, get_character_similarity
from pandas import Series, DataFrame
from util_spark import remove_stopwords_spark, detect_language_spark, flatten_list_of_tokens, spell_correct_tokens_spark, get_semantic_similarity_spark
from tokenization_spark import tokenize_sentence_nltk_spark
from pyspark.sql.functions import col
from pos_tagging_spark import run_treetagger_pos_tag_spark
from modeling_spark import run_word2vec_model_pyspark

In [5]:
# Reading input file(s) using python's default libraries
in_file = open("in_file.cfg").read()
in_file = in_file.split("\n")
patterns_file = in_file[5]
file_folder = in_file[4]
label = in_file[3]
column = in_file[2]
in_type = in_file[1]
in_file = in_file[0]
if file_folder == "file":
    strings = read_file(in_file, in_type = in_type)
    if in_type == "text":
        strings = tokenize_sentence_nltk(strings)
        strings = DataFrame(strings)[0]
    elif in_type == "html_chat":
        timestamp = strings[2]
        meta_data = strings[1]
        strings = strings[0]
        strings[label] = meta_data["Comment"]
        labels = strings[label]
        strings = strings[column]
    else:
        if label in strings.columns:
            labels = strings[label]
        strings = strings[column]
else:
    strings = read_folder(in_file, in_type = in_type)
    patterns = Series([".*" + x + ".*" for x in open(patterns_file, 'r').readlines()])

In [6]:
strings

,contents,meta_data
0,Dear All\nCan you pls. open the issue accordin...,"{'From': 'Chiappin, Hana (ZUPF 2)', 'Date': 'D..."
1,"Dear Ed\nPerfect, thank you. I will mail you a...","{'From': 'L, M (TXL 90)', 'Date': 'Freitag, 13..."
2,"Dear Monika,\nThank you for your email. 11am S...","{'From': 'L, E [mailto:Ed.ligere@company.com]'..."
3,Dear Ed\nKindly note Ying's availability as fo...,"{'From': 'L, M [mailto:m.laesser@domain.com]',..."
4,"Dear Ed,\nJanuary 24, 26 and 27 look good in m...","{'From': 'B, Y (TXC)', 'Sent': 'Donnerstag, 12..."
5,"Dear Ying,\nThank you for your email and conta...","{'From': 'L, E [mailto:Ed.ligere@company.com]'..."
6,">\n> Dear Ed,\n>\n> It was a pleasure to meet ...","{'From': 'Caiyilin, Joachim (C) >', 'Sent': 'D..."
7,">\n> Dear Joachim,\n>\n> I was delighted to me...","{'From': 'L, E [mailto:Ed.ligere@company.com] ..."


In [ ]:
# Appending conversation together and creating spark data frome
strings['conversation'] = strings['conversation'].apply(lambda x: ". ".join(x["Message"]))
sentenceDataFrame = spark.createDataFrame(strings)

In [ ]:
# Creating list of sentences for each conversation
sentenceDataFrame = tokenize_sentence_nltk_spark(df = sentenceDataFrame, in_col = "conversation")

In [ ]:
# Language identification and filtering
sentenceDataFrame = detect_language_spark(df = sentenceDataFrame, in_col = "conversation", out_col = "language")
sentenceDataFrame = sentenceDataFrame.where(col('language') == "en")

In [ ]:
# POS tagging and lemmatization using TreeTagger
sentenceDataFrame = run_treetagger_pos_tag_spark(df = sentenceDataFrame, in_col = "conversation", out_col = "pos", get_lemma = True)

In [ ]:
# Merging 2 consecutive words if a) Words are incorrectly spelled and b) Merged word is correctly spelled
sentenceDataFrame = spell_correct_tokens_spark(df = sentenceDataFrame, in_col = "pos")

In [ ]:
# Flattening out token of rows and running word2vec model
sentenceDataFrame = flatten_list_of_tokens(sentenceDataFrame, in_col = "pos")
model, sentenceDataFrame = run_word2vec_model_pyspark(sentenceDataFrame, in_col = "pos", vec_size = 100, in_type = "tokens", out_col = "result")

In [ ]:
# Collecting document vectors in a list
doc_vecs = []
for row in sentenceDataFrame.select('result').collect():
    doc_vecs = doc_vecs + [row['result']]

In [ ]:
sim1 = get_semantic_similarity_spark(model)

In [ ]:
sim1.head()

In [ ]:
ratio = get_character_similarity(sim1.columns, ratio_type = 'ratio')

In [ ]:
partial_ratio = get_character_similarity(sim1.columns, ratio_type = 'partial_ratio')

In [ ]:
token_sort_ratio = get_character_similarity(sim1.columns, ratio_type = 'token_sort_ratio')

In [ ]:
token_set_ratio = get_character_similarity(sim1.columns, ratio_type = 'token_set_ratio')

In [ ]:
sim1 = sim1[ratio.columns]
sim1 = sim1.loc[ratio.columns]

In [ ]:
semantic_weight = 0.5
ratio_weight = 0.4
partial_ratio_weight = 0.4
token_sort_ratio_weight = 0.1
sim = 1 - (semantic_weight*sim1 + (ratio_weight*ratio + partial_ratio_weight*partial_ratio + token_sort_ratio_weight*token_sort_ratio + (1-ratio_weight-partial_ratio_weight-token_sort_ratio_weight)*token_set_ratio)*(1-semantic_weight))